Загружаем набор данных


In [ ]:
import pandas as pd
import numpy as np

# Пример чтения файлов
movies_data = pd.read_csv('movies.dat', sep='::', encoding='latin1', engine='python', names=['MovieID', 'Title', 'Genres'])
users_data = pd.read_csv('users.dat', sep='::', encoding='latin1', engine='python', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
ratings_data = pd.read_csv('ratings.dat', sep='::', encoding='latin1', engine='python', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])


# Объедиенение данных
df = pd.merge(ratings_data, movies_data, on='MovieID')

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   UserID     1000209 non-null  int64 
 1   MovieID    1000209 non-null  int64 
 2   Rating     1000209 non-null  int64 
 3   Timestamp  1000209 non-null  int64 
 4   Title      1000209 non-null  object
 5   Genres     1000209 non-null  object
dtypes: int64(4), object(2)
memory usage: 53.4+ MB


Функция для создания матрицы пользователь_фильм

In [ ]:
def create_user_movie_matrix(df):
  user_movie_matrix = df.pivot_table(index='UserID', columns='MovieID', values='Rating', fill_value=0)
  return user_movie_matrix

Измерение сходства между пользователями реализуется с помощью косинусной меры или корреляции Пирсона.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Вычисляем мтарицу схожести, используя косинусную меру
def create_similarity_matrix_cosine(user_movie_matrix):
  similarity_matrix = cosine_similarity(user_movie_matrix)
  return similarity_matrix

# Вычисляем мтарицу схожести, используя корреляцию Пирсона
def create_similarity_matrix_pirson(user_movie_matrix):
  correlation_matrix = np.corrcoef(user_movie_matrix)
  return correlation_matrix

Функция выбора соседей, то есть пользователей, которые больше всего похожи на текущего пользователя

In [ ]:
def find_similar_users(user_id, n=5):
    user_similarity = similarity_matrix[user_id - 1]
    similar_users = np.argsort(user_similarity)[::-1][1:n+1]  # Исключаем самого пользователя из результата
    return similar_users

Функция генерации рекомендаций

In [ ]:
def recommend_movies(user_movie_matrix, similarity_matrix, user_id, n=5):
    user_ratings = user_movie_matrix.loc[user_id]
    similar_scores = similarity_matrix.dot(user_ratings)
    similar_movies = user_movie_matrix.columns[np.argsort(similar_scores)[::-1]]

    # Исключаем фильмы, которые пользователь уже оценил
    recommended_movies = [movie for movie in similar_movies if user_ratings[movie] == 0]

    return recommended_movies[:n]

In [ ]:
user_movie_matrix = create_user_movie_matrix(df)
similarity_matrix = create_similarity_matrix_cosine(user_movie_matrix)

# Пример рекомендации для пользователя с ID = ...
user_id = 15
recommendations = recommend_movies(user_movie_matrix, similarity_matrix, user_id)
print(f"Рекомендации для пользователя {user_id}: {recommendations}")

# Полный код
Реализация User-based Colloborative filtering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Пример чтения файлов
movies_data = pd.read_csv('movies.dat', sep='::', encoding='latin1', engine='python', names=['MovieID', 'Title', 'Genres'])
users_data = pd.read_csv('users.dat', sep='::', encoding='latin1', engine='python', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
ratings_data = pd.read_csv('ratings.dat', sep='::', encoding='latin1', engine='python', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

# Объединение данных
df = pd.merge(ratings_data, movies_data, on='MovieID')
df = df.replace(1, 0)

#print(df.info())


def create_user_movie_matrix(df):
    user_movie_matrix = df.pivot_table(index='UserID', columns='MovieID', values='Rating', fill_value=0)
    return user_movie_matrix


# Вычисляем матрицу схожести, используя косинусную меру
def create_similarity_matrix_cosine(user_movie_matrix):
    similarity_matrix = cosine_similarity(user_movie_matrix)
    return similarity_matrix

# Вычисляем матрицу схожести, используя корреляцию Пирсона
def create_similarity_matrix_pirson(user_movie_matrix):
    correlation_matrix = np.corrcoef(user_movie_matrix)
    return correlation_matrix

def find_similar_users(user_id, n=5, similarity_matrix=None):
    user_similarity = similarity_matrix[user_id - 1]
    similar_users = np.argsort(user_similarity)[::-1][1:n+1]  # Исключаем самого пользователя из результата
    return similar_users

def recommend_movies(user_id, user_movie_matrix, similarity_matrix, n=5):
    # Найти похожих пользователей
    similar_users = find_similar_users(user_id, n, similarity_matrix)

    # Найти фильмы, которые оценили похожие пользователи, но пользователь еще не смотрел
    user_movies = user_movie_matrix.loc[user_id]
    similar_users_movies = user_movie_matrix.loc[similar_users].sum()

    unrated_movies = similar_users_movies[user_movies == 0]

    # Рекомендовать топ N фильмов
    top_movies = unrated_movies.sort_values(ascending=False).head(n)

    # Получить названия рекомендованных фильмов
    recommended_movies = movies_data.loc[movies_data['MovieID'].isin(top_movies.index), 'Title']

    return recommended_movies

# Пример использования
user_movie_matrix = create_user_movie_matrix(df)
similarity_matrix = create_similarity_matrix_cosine(user_movie_matrix)
user_id_to_recommend = 8
recommended_movies = recommend_movies(user_id_to_recommend, user_movie_matrix, similarity_matrix)
print(f"Рекомендованные фильмы для пользователя {user_id_to_recommend}:\n{recommended_movies}")
print("-----------------------------------------------------------------------------------------")

def recommend_random_movie_with_genre(user_id, user_movie_matrix, movies_data, genre_to_avoid):
    # Получить фильмы, которые пользователь не оценил
    unrated_movies = user_movie_matrix.loc[user_id] == 0

    # Фильтровать фильмы по указанному жанру для избегания
    movies_to_avoid = movies_data[movies_data['Genres'].str.contains(genre_to_avoid)]
    unrated_movies_to_avoid = unrated_movies & unrated_movies.index.isin(movies_to_avoid['MovieID'])

    # Выбрать фильмы, которые пользователь не оценил и не принадлежат указанному жанру
    available_movies = unrated_movies[~unrated_movies_to_avoid]

    # Если есть доступные фильмы, рекомендовать случайный
    if available_movies.any():
        random_movie_id = np.random.choice(available_movies[available_movies].index)
        recommended_movie_info = movies_data[movies_data['MovieID'] == random_movie_id]
        recommended_movie_title = recommended_movie_info['Title'].values[0]
        recommended_movie_genre = recommended_movie_info['Genres'].values[0]
        return recommended_movie_title, recommended_movie_genre
    else:
        return "No available movies to recommend.", None

# Пример использования
user_id_to_recommend_random = 8
genre_to_avoid = 'Drama'  # Укажите жанр для избегания
recommended_movie_title, recommended_movie_genre = recommend_random_movie_with_genre(user_id_to_recommend_random, user_movie_matrix, movies_data, genre_to_avoid)

if recommended_movie_genre is not None:
    print(f"Рекомендованные фильмы для пользователя {user_id_to_recommend_random} (избегая жанр {genre_to_avoid}):\nTitle: {recommended_movie_title}\nGenre: {recommended_movie_genre}")
else:
    print(f"{recommended_movie_title}")



Рекомендованные фильмы для пользователя 8:
257             Star Wars: Episode IV - A New Hope (1977)
537                                   Blade Runner (1982)
908                             Gone with the Wind (1939)
1178    Star Wars: Episode V - The Empire Strikes Back...
1353                  Star Trek: The Wrath of Khan (1982)
Name: Title, dtype: object
-----------------------------------------------------------------------------------------
Рекомендованные фильмы для пользователя 8 (избегая жанр Drama):
Title: Air Bud (1997)
Genre: Children's|Comedy


------------------------------------------------------------------------------------------------

Item-based

In [ ]:
import pandas as pd
import numpy as np

# Пример чтения файлов
movies_data = pd.read_csv('movies.dat', sep='::', encoding='latin1', engine='python', names=['MovieID', 'Title', 'Genres'])
users_data = pd.read_csv('users.dat', sep='::', encoding='latin1', engine='python', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
ratings_data = pd.read_csv('ratings.dat', sep='::', encoding='latin1', engine='python', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

# Объединение данных
df = pd.merge(ratings_data, movies_data, on='MovieID')
df = df.replace(1, 0)

print(df.info())

def create_user_movie_matrix(df):
    user_movie_matrix = df.pivot_table(index='UserID', columns='MovieID', values='Rating', fill_value=0)
    return user_movie_matrix

def create_similarity_matrix_cosine(user_movie_matrix):
    # Транспонировать матрицу для вычисления схожести между фильмами
    movie_similarity_matrix = cosine_similarity(user_movie_matrix.T)
    return movie_similarity_matrix

def recommend_movies_item_based(movie_id, user_movie_matrix, similarity_matrix, n=5):
    # Найти похожие фильмы
    similar_movies = similarity_matrix[movie_id - 1]
    top_similar_movies = np.argsort(similar_movies)[::-1][1:n+1]

    # Получить названия рекомендованных фильмов
    recommended_movies = movies_data.loc[movies_data['MovieID'].isin(top_similar_movies), 'Title']

    return recommended_movies


# Пример использования для item-based рекомендации
user_movie_matrix = create_user_movie_matrix(df)
similarity_matrix = create_similarity_matrix_cosine(user_movie_matrix)
movie_id_to_recommend = 1  # Выберите MovieID, для которого хотите получить рекомендации
recommended_movies = recommend_movies_item_based(movie_id_to_recommend, user_movie_matrix, similarity_matrix)
print(f"Рекомендованные фильмы на основе фильма {movie_id_to_recommend}:\n{recommended_movies}")
print("-----------------------------------------------------------------------------------------")


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   UserID     1000209 non-null  int64 
 1   MovieID    1000209 non-null  int64 
 2   Rating     1000209 non-null  int64 
 3   Timestamp  1000209 non-null  int64 
 4   Title      1000209 non-null  object
 5   Genres     1000209 non-null  object
dtypes: int64(4), object(2)
memory usage: 53.4+ MB
None
Рекомендованные фильмы на основе фильма 1:
570                            Spanking the Monkey (1994)
1157      Cook the Thief His Wife & Her Lover, The (1989)
1162                                Paths of Glory (1957)
2093    NeverEnding Story II: The Next Chapter, The (1...
2829                                Dark Half, The (1993)
Name: Title, dtype: object
-----------------------------------------------------------------------------------------
